## Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

1. для групп normal (control) и early neoplasia (treatment)

2. для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [9]:
import pandas as pd
from scipy import stats

In [2]:
df = pd.read_csv('gene_high_throughput_sequencing.csv')

In [40]:
df['Diagnosis'].value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [43]:
df.columns[2:]

Index(['LOC643837', 'LOC100130417', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1',
       'C1orf170', 'HES4', 'ISG15', 'AGRN',
       ...
       'CLIC2', 'RPS4Y1', 'ZFY', 'PRKY', 'USP9Y', 'DDX3Y', 'CD24', 'CYorf15B',
       'KDM5D', 'EIF1AY'],
      dtype='object', length=15748)

In [49]:
t_norm_early = 0
t_early_cancer = 0
pvalues_norm_early = []
pvalues_early_cancer = []
for col in df.columns[2:]:
    pvalue_norm_early = stats.ttest_ind(df[df['Diagnosis']=='normal'][col],  
                    df[df['Diagnosis']=='early neoplasia'][col], 
                    equal_var =False)[1]
    
    pvalues_norm_early.append(pvalue_norm_early)
    
    if pvalue_norm_early < 0.05:
        t_norm_early += 1
    
    pvalue_early_cancer = stats.ttest_ind(df[df['Diagnosis']=='early neoplasia'][col],  
                    df[df['Diagnosis']=='cancer'][col], 
                    equal_var =False)[1]
    
    pvalues_early_cancer.append(pvalue_early_cancer)
    
    if pvalue_early_cancer < 0.05:
        t_early_cancer += 1

print(t_norm_early, t_early_cancer)

1575 3490


In [47]:
with open('1.txt', 'w') as f:
    f.write(str(t_norm_early))
with open('2.txt', 'w') as f:
    f.write(str(t_early_cancer))    
    

## Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

1
import statsmodels.stats.multitest as smm
В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.51.5.

Обратите внимание, что

применять поправку на множественную проверку  нужно  ко всем  значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.  

при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [57]:
import statsmodels.stats.multitest as smm
import numpy as np

In [51]:
pvalues_early_cancer
pvalues_norm_early

[0.6907660157497346,
 3.1785312845453036e-05,
 0.06027266393304932,
 0.8264285565676908,
 0.04987624508427378,
 0.14426667298693596,
 0.32910807600838166,
 0.023712385545179418,
 0.24014846249928945,
 0.03793281880327604,
 0.020001885854736043,
 0.009642375463674469,
 0.7103045087356202,
 0.17320482744649743,
 0.37791815514000016,
 0.15586878830972428,
 0.1002369911896769,
 0.006454311672213382,
 0.7486553239871663,
 0.13256681250190858,
 0.17418841285565928,
 0.5853850976673344,
 0.8100768971086594,
 0.9613520179500561,
 0.6371779848928885,
 0.6184568216605661,
 0.4325011376378963,
 0.586736542021224,
 0.7339163686148433,
 0.2465975151838163,
 0.22295965078712265,
 0.00978137990361677,
 0.48781243826896603,
 0.17589043756058292,
 0.42859885147200927,
 0.3989524155353815,
 0.9696088796030541,
 0.6990003405102847,
 0.09563016254395694,
 0.46518044880438714,
 0.10560768297930029,
 0.32211865898675673,
 0.4188586613380644,
 0.45806503384691544,
 0.6940459678446832,
 0.022970133954575693,


In [66]:
holmes_corr_pvalues_norm_early = smm.multipletests(pvalues_norm_early) # посчитал поправку Холма для пары "рак"/"ранняя стадия)
column_indexes = np.where(holmes_corr_pvalues_norm_early[1] < 0.025) # np.where() вернет мне массив индексов для значений массива, удовлетворяющих заданному условию

def column_names(array_of_indexes): # функция, возвращающая массив из имен колонок по их индексу
    names = []
    for i in array_of_indexes:
        names.append(list(df)[2:][i])
    return names

def fold_change(array1, array2): # простенькая функция, реализующая fold change
    mean1 = np.mean(array1)
    mean2 = np.mean(array2)
    if mean1 < mean2:
        return mean2/mean1
    else:
        return mean1/mean2

# теперь можно посчитать fold_changes для тех генов, значение p_value которых не удовлетворяет нашему условию. А потом тут же посчитать количество fold_changes, больших 1.5
counter = 0
for column in column_names(column_indexes[0]):
    if fold_change(df[df['Diagnosis']=='normal'][column], df[df['Diagnosis']=='early neoplasia'][column]) > 1.5:
        counter+=1
with open('3.txt', 'w') as f:
    f.write(str(counter))
    
holmes_corr_pvalues_early_cancer = smm.multipletests(pvalues_early_cancer) # посчитал поправку Холма для пары "рак"/"ранняя стадия)
column_indexes = np.where(holmes_corr_pvalues_early_cancer[1] < 0.025) # np.where() вернет мне массив индексов для значений массива, удовлетворяющих заданному условию
counter = 0
for column in column_names(column_indexes[0]):
    if fold_change(df[df['Diagnosis']=='early neoplasia'][column], df[df['Diagnosis']=='cancer'][column]) > 1.5:
        counter+=1
    
    
with open('4.txt', 'w') as f:
    f.write(str(counter))   

## Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

 Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H_0H 
0
​
 , когда они есть, и будут чаще отклонять H_0H 
0
​
 , когда отличий нет).

В качестве ответа к этому заданию требуется  ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 

In [70]:
p_fdr_bh_norm_early = smm.multipletests(pvalues_norm_early, 
                                            method = 'fdr_bh')[1]
p_fdr_bh_early_cancer = smm.multipletests(pvalues_early_cancer, 
                                            method = 'fdr_bh')[1]

column_indexes_norm_early = np.where(p_fdr_bh_norm_early < 0.025)
column_indexes_early_cancer = np.where(p_fdr_bh_early_cancer < 0.025)

counter = 0
for column in column_names(column_indexes_norm_early[0]):
    if fold_change(df[df['Diagnosis']=='normal'][column], df[df['Diagnosis']=='early neoplasia'][column]) > 1.5:
        counter+=1
with open('5.txt', 'w') as f:
    f.write(str(counter))
    
counter = 0
for column in column_names(column_indexes_early_cancer[0]):
    if fold_change(df[df['Diagnosis']=='early neoplasia'][column], df[df['Diagnosis']=='cancer'][column]) > 1.5:
        counter+=1
with open('6.txt', 'w') as f:
    f.write(str(counter))

In [69]:
p_fdr_bh_norm_early

array([0.96651052, 0.03569758, 0.53610276, ..., 0.96312033, 0.97735971,
       0.96074074])